<a href="https://colab.research.google.com/github/DaniellyBenicio/simulacao_revisao_espacada/blob/main/simulacao_chlr_x_ahlr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import truncnorm
from tqdm import tqdm
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import warnings
warnings.filterwarnings("ignore")

N_ROUNDS = 3
FIXED_SEED_INIT = 42
THRESHOLDS_P = [0.9, 0.7]
HORIZONTES_A_TESTAR = [15, 30, 60, 90, 120, 150, 180]


N_STUDENTS = 100
N_GROUP = 50

N_QUESTIONS = 500
N_SUBTEMAS = 5
SUBTEMAS = [f"S{i+1}" for i in range(N_SUBTEMAS)]
N_QUESTIONS_PER_SUBTEMA = N_QUESTIONS // N_SUBTEMAS

CARGA_PRATICA = 4
MIN_SPACING_INTERVAL = 10
H_MIN = 1.0
H_MAX = 365.0
K_ABILITY_UPDATE = 32
DIFFICULTY_RANGE_MARGIN = 150

# Fallbacks
FALLBACK_NONE = 'none'
FALLBACK_LEVEL1 = 'level1_urgency'
FALLBACK_LEVEL2 = 'level2_item_selection'
FB_REASON_OK = 'ok'
FB_REASON_NO_URGENT_SUBTOPIC = 'no_subtopic_below_threshold'
FB_REASON_NO_ITEMS_IN_RANGE = 'no_items_in_difficulty_range'
FB_REASON_MSI_BLOCKED = 'msi_blocked_all_candidates'
FB_REASON_USED_ALL_SUBTEMA = 'range_empty_used_all_subtema'

def get_truncated_normal(mean=500, sd=150, low=0, high=1000, size=None):
    return truncnorm((low-mean)/sd, (high-mean)/sd, loc=mean, scale=sd).rvs(size)

def fibonacci(n):
    if n <= 1: return 1
    a, b = 1, 1
    for _ in range(2, n+1):
        a, b = b, a + b
        if a > 1_000_000: return b
    return b

def simple_logistic_prob(ability, difficulty):
    return np.clip(1 / (1 + np.exp(-(ability - difficulty)/100)), 0.01, 0.99)

def generalized_probability(student, question, dt, h, h_cost=1.0):
    h_cost = compute_h_cost(student, question)
    base = simple_logistic_prob(student.ability, question.difficulty)
    retention = 0.5 ** (dt * h_cost / h)
    #if student.group == "G2":
    #print(f"Gr {student.group} | Aluno {student.ability} | Id_q {question.q_id} Dificuldade {question.difficulty} | h_cost {h_cost} | dt {dt} | h {h} |HLR {retention}")
    return np.clip(base * retention, 0.01, 0.99), retention, h_cost

def sample_with_overdispersion(p, kappa=50):
    alpha = max(p * kappa, 1e-6)
    beta = max((1-p) * kappa, 1e-6)
    p_pert = np.random.beta(alpha, beta)
    return 1 if np.random.random() < p_pert else 0

class Question:
    def __init__(self, q_id, subtema, difficulty):
        self.q_id = q_id
        self.subtema = subtema
        self.difficulty = difficulty
        self.last_day_seen = 0

class Student:
    def __init__(self, s_id, group, threshold, ability):
        self.s_id = s_id
        self.group = group
        self.threshold = threshold
        self.ability = ability
        #self.h_cost = 1.0 if group == "G1" else 1 - (ability/1000)
        self.h_cost = 1.0
        self.questions_state = {}
        self.subtema_h = {st: H_MIN for st in SUBTEMAS}
        self.subtema_last_seen = {st: 0 for st in SUBTEMAS}
        self.subtema_correct_streak = {st: 0 for st in SUBTEMAS}
        self.subtema_error_streak = {st: 0 for st in SUBTEMAS}

def update_half_life(q, student, result):
    st = q.subtema
    ratio = student.ability / (q.difficulty + 1e-6)
    h = student.subtema_h[st]
    if result == 1:
        student.subtema_correct_streak[st] += 1
        student.subtema_error_streak[st] = 0
        inc = ratio * fibonacci(student.subtema_correct_streak[st])
        h_new = h + inc
    else:
        student.subtema_error_streak[st] += 1
        student.subtema_correct_streak[st] = 0
        inc = (q.difficulty + 1e-6) / (student.ability + 1e-6) * fibonacci(student.subtema_error_streak[st])
        h_new = H_MIN + inc / 10
    student.subtema_h[st] = np.clip(h_new, H_MIN, H_MAX)

def compute_h_cost(student, question):
    if student.group == "G1":
        return question.difficulty / 1000
    else:
        return 1 - (student.ability / 1000)

def update_ability(student, q, result):
    p_exp = simple_logistic_prob(student.ability, q.difficulty)
    corr = K_ABILITY_UPDATE * (result - p_exp)
    old = student.ability
    student.ability = np.clip(student.ability + corr, 0, 1000)
    h_cost = compute_h_cost(student, q)
    student.h_cost = h_cost
    #if student.group == "G2":
        #student.h_cost = 1 - (student.ability / 1000)
    return corr, p_exp, h_cost, old

def select_question(student, day):
    if not student.questions_state:
        return None, 0, True, FALLBACK_LEVEL1, "no_state", 0, 0

    # --- Nível 1: urgência ---
    urgencies = {}
    for st in SUBTEMAS:
        dt = day - student.subtema_last_seen[st]
        h = student.subtema_h[st]
        p_ret = 0.5 ** (dt * student.h_cost / h)
        if p_ret < student.threshold:
            urgencies[st] = p_ret

    if not urgencies:
        return None, 0, True, FALLBACK_LEVEL1, FB_REASON_NO_URGENT_SUBTOPIC, 0, 0

    chosen_st = min(urgencies, key=urgencies.get)

    # --- Candidatos ideais (com faixa e MSI) ---
    candidates = [q for q in student.questions_state.values()
                  if q.subtema == chosen_st
                  and abs(q.difficulty - student.ability) <= DIFFICULTY_RANGE_MARGIN
                  and (q.last_day_seen == 0 or day - q.last_day_seen >= MIN_SPACING_INTERVAL)]

    is_fb = False
    fb_stage = FALLBACK_NONE
    reason = FB_REASON_OK

    if not candidates:
        # Fallback nível 2
        candidates = [q for q in student.questions_state.values() if q.subtema == chosen_st]
        if not candidates:
            return None, 0, True, FALLBACK_LEVEL2, FB_REASON_NO_ITEMS_IN_RANGE, 0, 0

        is_fb = True
        fb_stage = FALLBACK_LEVEL2
        reason = FB_REASON_USED_ALL_SUBTEMA

    # Escolhe a melhor questão entre as disponíveis
    best_q = min(candidates, key=lambda q: abs(simple_logistic_prob(student.ability, q.difficulty) - 0.5))

    dt = day - student.subtema_last_seen[chosen_st]
    h = student.subtema_h[chosen_st]
    p_est, retention, h_cost = generalized_probability(student, best_q, dt, h, student.h_cost)

    return best_q, p_est, is_fb, fb_stage, reason, retention, h_cost

all_sim_data = []
all_post_data = []

print("INICIANDO SIMULAÇÃO:\n")

for horizonte in HORIZONTES_A_TESTAR:
    print(f"Horizonte: {horizonte} dias")

    np.random.seed(FIXED_SEED_INIT)
    difficulties = get_truncated_normal(size=N_QUESTIONS)
    subthemes = np.repeat(SUBTEMAS, N_QUESTIONS_PER_SUBTEMA)
    np.random.shuffle(subthemes)
    questions_pool = {i+1: Question(i+1, subthemes[i], difficulties[i]) for i in range(N_QUESTIONS)}

    abilities = get_truncated_normal(size=N_STUDENTS)

    if horizonte == HORIZONTES_A_TESTAR[0]:
        plt.figure(figsize=(12,5))
        plt.subplot(1,2,1)
        sns.histplot(abilities, bins=20, kde=True, color='skyblue')
        plt.title('Distribuição Inicial de Habilidade dos Alunos')
        plt.subplot(1,2,2)
        sns.histplot(difficulties, bins=20, kde=True, color='salmon')
        plt.title('Distribuição Inicial de Dificuldade das Questões')
        plt.tight_layout()
        plt.show()

    for threshold in THRESHOLDS_P:
        for run in range(1, N_ROUNDS+1):
            np.random.seed(FIXED_SEED_INIT + run*1000 + int(threshold*100))

            students = []
            for i in range(N_STUDENTS):
                group = "G1" if i < N_GROUP else "G2"
                s = Student(i+1, group, threshold, abilities[i])
                s.questions_state = {q.q_id: Question(q.q_id, q.subtema, q.difficulty) for q in questions_pool.values()}
                students.append(s)

            logs = []
            for day in tqdm(range(1, horizonte+1), desc=f"p={threshold} Run {run}", leave=False):
                for student in students:
                  praticadas_no_dia = 0

                  while praticadas_no_dia < CARGA_PRATICA:
                    q, p_est, is_fb, fb_stage, fb_reason, retention, h_cost = select_question(student, day)
                    if q is None and fb_stage == FALLBACK_LEVEL1:
                        break

                    if q is None:
                        continue

                    dt = day - student.subtema_last_seen[q.subtema]
                    h_dpg = student.subtema_h[q.subtema]
                    p_acerto = p_est #generalized_probability(student, q, dt, h_dpg, student.h_cost)

                    result = sample_with_overdispersion(p_acerto)

                    init_ability = student.ability
                    init_correct = student.subtema_correct_streak[q.subtema]
                    init_error = student.subtema_error_streak[q.subtema]

                    corr, p_update, h_cost, _ = update_ability(student, q, result)
                    update_half_life(q, student, result)

                    final_correct = student.subtema_correct_streak[q.subtema]
                    final_error = student.subtema_error_streak[q.subtema]

                    student.subtema_last_seen[q.subtema] = day
                    q.last_day_seen = day

                    logs.append({
                        'q_difficulty': q.difficulty,
                        'h_cost': h_cost,
                        'C ou A-HLR': retention,

                        'run': run, 'horizonte_dias': horizonte, 'day': day,
                        'group': student.group, 'p_limiar': threshold,
                        'student_id': student.s_id,
                        'ability_start': init_ability, 'ability_correction': corr,
                        'ability_end': student.ability, 'prob_update_model': p_update,
                        'h_cost_end': student.h_cost,
                        'is_fallback': is_fb, 'fallback_stage': fb_stage, 'fallback_reason': fb_reason,
                        'q_id': q.q_id, 'q_subtema': q.subtema, 'q_difficulty': q.difficulty,
                        'result': result, 'p_acerto_real': p_acerto, 'p_est_revisao': p_est,
                        'h_do_subtema': h_dpg, 'h_final': student.subtema_h[q.subtema],
                        'dt': dt,
                        'q_correct_start': init_correct, 'q_error_start': init_error,
                        'q_correct_end': final_correct, 'q_error_end': final_error,
                    })
                    praticadas_no_dia += 1

            df_run = pd.DataFrame(logs)
            df_run['algoritmo'] = df_run.apply(lambda x: f"{x.group} p={x.p_limiar}", axis=1)
            all_sim_data.append(df_run)

            # Pós-teste
            post = []
            test_day = horizonte + 1
            for student in students:
                for q in student.questions_state.values():
                    dt = test_day - student.subtema_last_seen[q.subtema]
                    h = student.subtema_h[q.subtema]
                    p, retention, h_cost = generalized_probability(student, q, dt, h, student.h_cost)
                    res = 1 if np.random.rand() < p else 0
                    post.append({
                        'run': run, 'horizonte_dias': horizonte,
                        'group': student.group, 'p_limiar': threshold,
                        'algoritmo': f"{student.group} p={threshold}",
                        'student_id': student.s_id, 'result': res
                    })
            all_post_data.append(pd.DataFrame(post))

#  RESULTADOS FINAIS
df_sim = pd.concat(all_sim_data, ignore_index=True)
df_post = pd.concat(all_post_data, ignore_index=True)

df_sim['algoritmo'] = df_sim['group'] + ' p=' + df_sim['p_limiar'].astype(str)
df_post['algoritmo'] = df_post['group'] + ' p=' + df_post['p_limiar'].astype(str)

# Salvar CSVs
df_sim.to_csv("SIMULACAO_G1_HLR_vs_G2_AHLR_COMPLETA.csv", index=False, encoding="utf-8")
df_post.to_csv("POSTTEST_G1_HLR_vs_G2_AHLR_COMPLETA.csv", index=False, encoding="utf-8")


#  GRÁFICOS E ANÁLISES
def plot_metrics_by_horizon(df, title, y_label, metric):
    #plt.figure(figsize=(15, 8))
    plt.figure(figsize=(20, 10), dpi=300)
    df_agg = df.groupby(['run', 'algoritmo', 'horizonte_dias'])[metric].mean().reset_index()
    df_agg['grupo_completo'] = df_agg['algoritmo'] + ' (H:' + df_agg['horizonte_dias'].astype(str) + ')'
    sns.barplot(x='grupo_completo', y=metric, data=df_agg, hue='horizonte_dias', palette='viridis', errorbar=('ci', 95))
    plt.title(title)
    plt.ylabel(y_label + ' (Média das Rodadas com IC 95%)')
    plt.xlabel('Algoritmo de Agendamento')
    plt.xticks(rotation=45, ha='right')
    plt.legend(title='Horizonte (Dias)')
    plt.grid(axis='y', linestyle='--')
    plt.tight_layout()
    plt.show()

df_acc = (
    df_sim
    .groupby(['run', 'algoritmo', 'horizonte_dias'])['result']
    .mean()
    .reset_index(name='accuracy')
)

plot_metrics_by_horizon(
    df_sim,
    'Taxa de Acerto Média DURANTE a Prática',
    'Taxa de Acerto',
    'result'
)

plot_metrics_by_horizon(
    df_post,
    'Retenção Final (Dia N+1) - Taxa de Acerto no Pós-Teste',
    'Taxa de Acerto no Pós-Teste',
    'result'
)


def plot_g1_vs_g2_by_horizon(df, title, y_label, metric):
    plt.figure(figsize=(14, 7), dpi=300)

    df_agg = (
        df.groupby(['run', 'group', 'horizonte_dias'])[metric]
          .mean()
          .reset_index()
    )

    ax = sns.barplot(
        data=df_agg,
        x='horizonte_dias',
        y=metric,
        hue='group',
        errorbar=('ci', 95),
        palette='Set2'
    )

    for p in ax.patches:
        height = p.get_height()
        if not np.isnan(height):
            ax.annotate(
                f'{height*100:.1f}%',
                (p.get_x() + p.get_width() / 2., height),
                ha='center',
                va='bottom',
                fontsize=9,
                xytext=(0, 4),
                textcoords='offset points'
            )

    plt.title(title)
    plt.ylabel(y_label + ' (%)')
    plt.xlabel('Horizonte de Previsão (dias)')
    plt.legend(title='Algoritmo')
    plt.grid(axis='y', linestyle='--')
    plt.tight_layout()
    plt.show()


df_sim_p07 = df_sim[df_sim['p_limiar'] == 0.7]
df_sim_p09 = df_sim[df_sim['p_limiar'] == 0.9]

plot_g1_vs_g2_by_horizon(
    df_sim_p07,
    'Taxa de Acerto Média DURANTE a Prática — G1 vs G2 (p = 0.7)',
    'Taxa de Acerto',
    'result'
)

plot_g1_vs_g2_by_horizon(
    df_sim_p09,
    'Taxa de Acerto Média DURANTE a Prática — G1 vs G2 (p = 0.9)',
    'Taxa de Acerto',
    'result'
)

def plot_linhas_g1_g2(df_acc, p_valor, titulo):
    df_plot = df_acc[df_acc['algoritmo'].str.contains(f'p={p_valor}')].copy()

    df_plot['grupo'] = df_plot['algoritmo'].str.extract(r'(G\d)')

    plt.figure(figsize=(10, 6), dpi=300)

    sns.lineplot(
        data=df_plot,
        x='horizonte_dias',
        y='accuracy',
        hue='grupo',
        marker='o',
        errorbar=('ci', 95)
    )

    plt.title(titulo)
    plt.xlabel('Horizonte (dias)')
    plt.ylabel('Taxa de Acerto Média')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.legend(title='Grupo')
    plt.tight_layout()
    plt.show()


plot_linhas_g1_g2(
    df_acc,
    p_valor=0.7,
    titulo='Evolução da Taxa de Acerto Durante a Prática (p = 0.7)'
)

plot_linhas_g1_g2(
    df_acc,
    p_valor=0.9,
    titulo='Evolução da Taxa de Acerto Durante a Prática (p = 0.9)'
)

df_acc = (
    df_sim
    .groupby(['run', 'group', 'p_limiar', 'horizonte_dias'])['result']
    .mean()
    .reset_index(name='accuracy')
)

plt.figure(figsize=(12, 7), dpi=300)

sns.lineplot(
    data=df_acc,
    x='horizonte_dias',
    y='accuracy',
    hue='group',
    style='p_limiar',
    markers=True,
    dashes=True,
    errorbar=('ci', 95)
)

plt.title('Evolução da Taxa de Acerto — G1 vs G2 para p = 0.7 e p = 0.9')
plt.xlabel('Horizonte (dias)')
plt.ylabel('Taxa de Acerto Média')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(title='Grupo / Limiar')
plt.tight_layout()
plt.show()

# RETENÇÃO FINAL

def plot_retencao_por_horizonte(df_post, p_valor, titulo):
    df_plot = (
        df_post[df_post['p_limiar'] == p_valor]
        .groupby(['run', 'group', 'horizonte_dias'])['result']
        .mean()
        .reset_index()
    )

    plt.figure(figsize=(10, 6), dpi=300)

    sns.barplot(
        data=df_plot,
        x='horizonte_dias',
        y='result',
        hue='group',
        errorbar=('ci', 95),
        palette='Set2'
    )

    plt.title(titulo)
    plt.xlabel('Horizonte de Revisão (dias)')
    plt.ylabel('Taxa de Acerto no Pós-Teste')
    plt.legend(title='Algoritmo')
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()


plot_retencao_por_horizonte(
    df_post,
    p_valor=0.7,
    titulo='Retenção Final (Dia N+1) — G1 vs G2 (p = 0.7)'
)

plot_retencao_por_horizonte(
    df_post,
    p_valor=0.9,
    titulo='Retenção Final (Dia N+1) — G1 vs G2 (p = 0.9)'
)


# Fallback
print("\n--- Frequência Média de Agendamento por Fallback (Indica Saturação) ---")
df_fb = df_sim.groupby(['run', 'algoritmo', 'horizonte_dias'])['is_fallback'].mean().reset_index()
df_fb['porcentagem'] = df_fb['is_fallback'] * 100
df_fb['grupo_h'] = df_fb['algoritmo'] + ' (H:' + df_fb['horizonte_dias'].astype(str) + ')'
plt.figure(figsize=(14, 7))
sns.barplot(x='grupo_h', y='porcentagem', hue='horizonte_dias', data=df_fb, palette='magma', errorbar=('ci', 95))
plt.title('Frequência de Fallback (%)')
plt.ylabel('Fallbacks (%)')
plt.xlabel('Algoritmo e Horizonte')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

# Carga por subtema
print("\n--- Distribuição da Carga de Revisão por Subtema ---")
df_carga_por_aluno = df_sim.groupby(['run', 'algoritmo', 'horizonte_dias', 'student_id', 'q_subtema']).size().reset_index(name='respostas_aluno')
df_carga_agg = df_carga_por_aluno.groupby(['run', 'algoritmo', 'horizonte_dias', 'q_subtema'])['respostas_aluno'].mean().reset_index(name='media_por_aluno')
df_total = df_carga_agg.groupby(['run', 'algoritmo', 'horizonte_dias'])['media_por_aluno'].sum().reset_index()
df_total['q_subtema'] = 'Total Geral'
df_carga_full = pd.concat([df_carga_agg, df_total], ignore_index=True)
df_carga_final = df_carga_full.groupby(['algoritmo', 'q_subtema', 'horizonte_dias'])['media_por_aluno'].mean().reset_index()
df_carga_final['algoritmo_h'] = df_carga_final['algoritmo'] + ' (H:' + df_carga_final['horizonte_dias'].astype(str) + ')'

plt.figure(figsize=(16, 8))
sns.barplot(x='q_subtema', y='media_por_aluno', hue='algoritmo_h',
            data=df_carga_final[df_carga_final['q_subtema'] != 'Total Geral'], palette='Spectral')
plt.title('Distribuição Média de Revisões por Subtema (por aluno)')
plt.ylabel('Média de Respostas por Aluno')
plt.xlabel('Subtema')
plt.legend(title='Algoritmo e Horizonte', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(axis='y', linestyle='--')
plt.tight_layout()
plt.show()

print("\n[TABELA DE CARGA DE REVISÃO — Média de Respostas por Aluno]")
df_pivot = df_carga_final[df_carga_final['q_subtema'].isin(SUBTEMAS + ['Total Geral'])].pivot_table(
    index=['horizonte_dias', 'q_subtema'], columns='algoritmo', values='media_por_aluno', fill_value=0).round(2)
print(df_pivot)

# Análise estatística
print("\n" + "="*60)
print("ANÁLISE ESTATÍSTICA FORMAL")
print("="*60)

df_acc = df_sim.groupby(['run', 'algoritmo', 'horizonte_dias'])['result'].mean().reset_index(name='accuracy')
df_acc['combo'] = df_acc['algoritmo'] + '_H' + df_acc['horizonte_dias'].astype(str)
anova_acc = stats.f_oneway(*[g['accuracy'] for _, g in df_acc.groupby('combo')])
print(f"\nTaxa de Acerto na Prática → ANOVA: F={anova_acc.statistic:.2f}, p={anova_acc.pvalue:.2e}")
if anova_acc.pvalue < 0.05:
    tukey_acc = pairwise_tukeyhsd(df_acc['accuracy'], df_acc['combo'])
    print(tukey_acc)

df_ret = df_post.groupby(['run', 'algoritmo', 'horizonte_dias'])['result'].mean().reset_index(name='retention')
df_ret['combo'] = df_ret['algoritmo'] + '_H' + df_ret['horizonte_dias'].astype(str)
anova_ret = stats.f_oneway(*[g['retention'] for _, g in df_ret.groupby('combo')])
print(f"\nRetenção Final (Pós-teste) → ANOVA: F={anova_ret.statistic:.2f}, p={anova_ret.pvalue:.2e}")
if anova_ret.pvalue < 0.05:
    tukey_ret = pairwise_tukeyhsd(df_ret['retention'], df_ret['combo'])
    print(tukey_ret)

# Repetições
print("\n--- Total de Repetições de Questões (Interações > 1) ---")
df_freq = df_sim.groupby(['run', 'algoritmo', 'horizonte_dias', 'student_id', 'q_id']).size().reset_index(name='freq')
df_freq['repeticoes'] = df_freq['freq'] - 1
df_rep = df_freq.groupby(['run', 'algoritmo', 'horizonte_dias'])['repeticoes'].sum().reset_index()
df_rep['media_por_aluno'] = df_rep['repeticoes'] / N_GROUP
df_rep_final = df_rep.groupby(['algoritmo', 'horizonte_dias'])['media_por_aluno'].mean().reset_index()
df_rep_pivot = df_rep_final.pivot(index='horizonte_dias', columns='algoritmo', values='media_por_aluno').round(2)
print("\nMédia de Repetições por Aluno:")
print(df_rep_pivot)

#  FALLBACK

print("\n--- Frequência Média de Agendamento por Fallback (Indica Saturação) ---")

df_fb = (
    df_sim
    .groupby(['run', 'group', 'p_limiar', 'horizonte_dias'])['is_fallback']
    .mean()
    .reset_index()
)

df_fb['porcentagem'] = df_fb['is_fallback'] * 100

df_fb_p07 = df_fb[df_fb['p_limiar'] == 0.7]

plt.figure(figsize=(10, 6), dpi=300)

sns.barplot(
    data=df_fb_p07,
    x='horizonte_dias',
    y='porcentagem',
    hue='group',
    errorbar=('ci', 95),
    palette='Set2'
)

plt.title('Frequência de Fallback (%) — p = 0.7')
plt.xlabel('Horizonte (dias)')
plt.ylabel('Fallback (%)')
plt.legend(title='Algoritmo')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

df_fb_p09 = df_fb[df_fb['p_limiar'] == 0.9]

plt.figure(figsize=(10, 6), dpi=300)

sns.barplot(
    data=df_fb_p09,
    x='horizonte_dias',
    y='porcentagem',
    hue='group',
    errorbar=('ci', 95),
    palette='Set2'
)

plt.title('Frequência de Fallback (%) — p = 0.9')
plt.xlabel('Horizonte (dias)')
plt.ylabel('Fallback (%)')
plt.legend(title='Algoritmo')
plt.grid(axis='y', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

print("\n--- Evolução Temporal das Métricas ---")
plt.figure(figsize=(12, 6))
sns.lineplot(data=df_acc, x='horizonte_dias', y='accuracy', hue='algoritmo', marker='o', errorbar=('ci', 95))
plt.title('Evolução da Taxa de Acerto Durante a Prática')
plt.xlabel('Horizonte (dias)')
plt.ylabel('Taxa de Acerto Média')
plt.grid(True, linestyle='--')
plt.legend(title='Algoritmo')
plt.show()

plt.figure(figsize=(12, 6))
sns.lineplot(data=df_ret, x='horizonte_dias', y='retention', hue='algoritmo', marker='o', errorbar=('ci', 95))
plt.title('Evolução da Retenção no Pós-Teste')
plt.xlabel('Horizonte (dias)')
plt.ylabel('Taxa de Retenção Média')
plt.grid(True, linestyle='--')
plt.legend(title='Algoritmo')
plt.show()

# EXPORTAÇÃO FINAL
print("\n" + "="*80)
print("SALVANDO ARQUIVOS FINAIS...")
print("="*80)

excel_filename = "RESULTADOS_COMPLETOS_HLR_vs_AHLR.xlsx"
with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
    df_sim.to_excel(writer, sheet_name="Simulacao_Diaria", index=False)
    #df_post.to_excel(writer, sheet_name="Post_Teste", index=False)

    summary_acc = df_sim.groupby(['algoritmo', 'horizonte_dias'])['result'].mean().reset_index()
    summary_acc.rename(columns={'result': 'Taxa_Acerto_Pratica'}, inplace=True)
    summary_ret = df_post.groupby(['algoritmo', 'horizonte_dias'])['result'].mean().reset_index()
    summary_ret.rename(columns={'result': 'Taxa_Retencao_PosTeste'}, inplace=True)
    summary = pd.merge(summary_acc, summary_ret, on=['algoritmo', 'horizonte_dias'], how='outer').round(4)
    summary.to_excel(writer, sheet_name="Resumo_Metricas", index=False)

    df_pivot.to_excel(writer, sheet_name="Carga_por_Subtema", index=True)
    df_rep_pivot.to_excel(writer, sheet_name="Repeticoes_por_Aluno", index=True)

print(f"Arquivos salvos com sucesso!")
print(f"   → SIMULACAO_G1_HLR_vs_G2_AHLR_COMPLETA.csv")
print(f"   → POSTTEST_G1_HLR_vs_G2_AHLR_COMPLETA.csv")
print(f"   → {excel_filename}")
print(f"     Abas: Simulacao_Diaria | Post_Teste | Resumo_Metricas | Carga_por_Subtema | Repeticoes_por_Aluno")
print("="*80)


